In [ ]:
import pandas as pd
import meterdatalogic as ml

TZ = "Australia/Brisbane"


In [ ]:
# 7 days of half-hourly data for one NMI
rng = pd.date_range("2025-01-01", periods=48*7, freq="30min", tz=TZ)
df = pd.DataFrame({
    "t_start": rng,
    "nmi": "Q1234567890",
    "channel": "E1",
    "kwh": 0.5
})
df.head()


In [ ]:
canon_df = ml.ingest.from_nem12("../data/Sample1.csv", tz="Australia/Brisbane")
ml.validate.assert_canon(canon_df)

print(canon_df.index.tz, df.index.name)
print(canon_df.head())
print(canon_df["flow"].value_counts())
canon_df

In [ ]:
# canon_df = ml.ingest.from_dataframe(df, tz=TZ)
# ml.validate.assert_canon(canon_df)

# print("✅ Canonicalized columns:", canon_df.columns.tolist())
# canon_df.head(3)


In [ ]:
daily = ml.transform.groupby_day(canon_df)
daily.head()


In [ ]:
monthly = ml.transform.groupby_month(canon_df)
monthly.head()


In [ ]:
profile24 = ml.transform.profile24(canon_df)
profile24.head()


In [ ]:
bands = [
    {"name": "off", "start": "00:00", "end": "16:00"},
    {"name": "peak", "start": "16:00", "end": "21:00"},
    {"name": "shoulder", "start": "21:00", "end": "24:00"},
]
tou = ml.transform.tou_bins(canon_df, bands)
tou.head()


In [ ]:
demand = ml.transform.demand_window(canon_df, start="16:00", end="21:00", days="MF")
demand.head()


In [ ]:
summary = ml.summary.summarise(canon_df)
summary["meta"], summary["energy"], summary["per_day_avg_kwh"]


In [ ]:
import json
print(json.dumps(summary, indent=2)[:800])


In [ ]:
# Define a simple ToU plan
plan = ml.types.Plan(
    usage_bands=[
        ml.types.ToUBand("off", "00:00", "16:00", 22.0),
        ml.types.ToUBand("peak", "16:00", "21:00", 45.0),
        ml.types.ToUBand("shoulder", "21:00", "24:00", 28.0),
    ],
    demand=ml.types.DemandCharge("16:00", "21:00", "MF", 12.0),
    fixed_c_per_day=95.0,
    feed_in_c_per_kwh=6.0
)

monthly_cost = ml.pricing.estimate_monthly_cost(canon_df, plan)
monthly_cost.head()


In [ ]:
payload = {
    "summary": ml.summary.summarise(canon_df),
    "daily": ml.transform.groupby_day(canon_df).reset_index().to_dict("records"),
    "monthly": ml.transform.groupby_month(canon_df).to_dict("records"),
    "profile24": ml.transform.profile24(canon_df).to_dict("records"),
    "tou": ml.transform.tou_bins(canon_df, bands).to_dict("records"),
    "cost": ml.pricing.estimate_monthly_cost(canon_df, plan).to_dict("records")
}

# Quick preview
import json
print(json.dumps(payload["summary"], indent=2)[:800])
